<a href="https://colab.research.google.com/github/Franciscotor1/PokemonSpark-cards-DB-1999-2023/blob/master/Pkemonesbd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instala Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Descarga e instala Apache Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

# Configura las variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

# Añade Spark al sistema
import findspark
findspark.init()

# Crea una sesión Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("example").getOrCreate()


# Nueva sección

In [2]:
spark.version

'3.1.2'

In [3]:
# Instala e importa las bibliotecas necesarias
!pip install pyspark
from pyspark.sql import SparkSession

# Inicia una sesión de Spark
spark = SparkSession.builder.appName("pokemon-analysis").getOrCreate()

# Lee el conjunto de datos desde el archivo CSV
file_path = "pokemondata-1999-2023.csv"  # Asegúrate de que el archivo esté en la misma ubicación que este script o proporciona la ruta completa
pokemon_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Muestra el esquema del DataFrame
pokemon_df.printSchema()

# Muestra las primeras filas del DataFrame
pokemon_df.show(5)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=ab79b218b320cb01634639452c433269978f4f27d9123b7a0c417eba3b60dfcb
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


root
 |-- id: string (nullable = true)
 |-- set: string (nullable = true)
 |-- series: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- generation: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- name: string (nullable = true)
 |-- set_num: string (nullable = true)
 |-- types: string (nullable = true)
 |-- supertype: string (nullable = true)
 |-- subtypes: string (nullable = true)
 |-- level: string (nullable = true)
 |-- hp: integer (nullable = true)
 |-- evolvesFrom: string (nullable = true)
 |-- evolvesTo: string (nullable = true)
 |-- abilities: string (nullable = true)
 |-- attacks: string (nullable = true)
 |-- weaknesses: string (nullable = true)
 |-- retreatCost: string (nullable = true)
 |-- convertedRetreatCost: string (nullable = true)
 |-- rarity: string (nullable = true)
 |-- flavorText: string (nullable = true)
 |-- nationalPokedexNumbers: string (nullable = true)
 |-- legalities: string

In [4]:
from pyspark.sql.functions import col

pokemon_count_by_generation = pokemon_df.groupBy("generation").count()
pokemon_count_by_generation.show()


+----------+-----+
|generation|count|
+----------+-----+
|    Fourth| 1438|
|   Seventh| 2973|
|     First|  758|
|    Eighth| 3665|
|     Other|  157|
|    Second| 1031|
|   Fourth |  592|
|     Ninth| 1304|
|     Sixth| 1710|
|     Fifth| 1653|
|     Third| 1891|
+----------+-----+



In [5]:
max_hp_pokemon = pokemon_df.orderBy(col("hp").desc()).first()
print("El Pokémon con la mayor cantidad de HP es:", max_hp_pokemon["name"], "con", max_hp_pokemon["hp"], "HP")


El Pokémon con la mayor cantidad de HP es: Eternatus VMAX con 340 HP


In [6]:
# Calcular el promedio de nivel por rareza DE POKEMON
avg_level_by_rarity = pokemon_df.groupBy("rarity").agg({"level": "avg"})
avg_level_by_rarity.show()

+--------------------+------------------+
|              rarity|        avg(level)|
+--------------------+------------------+
| put Deoxys on to...|              null|
|[{'name': 'Upward...|              null|
| {'name': 'Thunde...|              37.0|
| 'name': 'Super P...|              null|
| 'type': 'Pokémon...|29.555555555555557|
|          Rare Prime|              null|
| {'name': 'Electr...|              null|
| {'name': 'Combus...|              null|
|[{'name': 'Chomp'...|              56.0|
|"[{'name': 'Venge...|              null|
| put Castform on ...|              35.0|
| 'convertedEnergy...|              null|
|  'cost': ['Psychic'|36.166666666666664|
|   'cost': ['Grass']|              15.0|
| {'name': 'Brutal...|              52.0|
|"[{'name': 'Hydro...|              52.0|
| done to Seaking....|              null|
|[{'name': 'Poison...|              null|
| or anything else...|              37.0|
|['Colorless', 'Co...|43.333333333333336|
+--------------------+------------

In [7]:
# Filtrar Pokémon que tienen una habilidad específica
pokemon_with_ability = pokemon_df.filter(col("abilities").contains("your_specific_ability"))
pokemon_with_ability.show()

+---+---+------+---------+----------+------------+------+----+-------+-----+---------+--------+-----+---+-----------+---------+---------+-------+----------+-----------+--------------------+------+----------+----------------------+----------+-----------+-----+--------------+------------+
| id|set|series|publisher|generation|release_date|artist|name|set_num|types|supertype|subtypes|level| hp|evolvesFrom|evolvesTo|abilities|attacks|weaknesses|retreatCost|convertedRetreatCost|rarity|flavorText|nationalPokedexNumbers|legalities|resistances|rules|regulationMark|ancientTrait|
+---+---+------+---------+----------+------------+------+----+-------+-----+---------+--------+-----+---+-----------+---------+---------+-------+----------+-----------+--------------------+------+----------+----------------------+----------+-----------+-----+--------------+------------+
+---+---+------+---------+----------+------------+------+----+-------+-----+---------+--------+-----+---+-----------+---------+---------

In [9]:
# Imprimir la lista única de habilidades
unique_abilities = pokemon_df.select("abilities").distinct().collect()
for row in unique_abilities:
    print(row['abilities'])

"[{'name': 'Ripples', 'text': ""Once during your turn (before your attack)
"[{'name': 'Dark Healer', 'text': ""As long as Rocket's Snorlax ex has any Darkness Energy attached to it
[{'name': 'Baby Evolution', 'text': 'Once during your turn (before your attack), you may put Jynx from your hand onto Smoochum (this counts as evolving Smoochum) and remove all damage counters from Smoochum.', 'type': 'Poké-Power'}]
"[{'name': 'Bedhead', 'text': 'As long as Snorlax remains Asleep between turns, put 2 damage counters on 1 of the Defending Pokémon.', 'type': 'Poké-Body'}, {'name': 'Dozing', 'text': ""Once during your turn (before your attack)
"[{'name': 'Grass Whistle', 'text': ""Once during your turn (before your attack)
"[{'name': 'Magma Armor', 'text': ""Magcargo can't be Asleep or Paralyzed.""
[{'name': 'Energy Refresh', 'text': 'As long as Leafeon is your Active Pokémon, whenever you attach an Energy card from your hand to 1 of your Pokémon, remove 2 damage counters from that Pokémon.', '

In [14]:
pokemon_df.groupBy("generation").count().show()


+----------+-----+
|generation|count|
+----------+-----+
|    Fourth| 1438|
|   Seventh| 2973|
|     First|  758|
|    Eighth| 3665|
|     Other|  157|
|    Second| 1031|
|   Fourth |  592|
|     Ninth| 1304|
|     Sixth| 1710|
|     Fifth| 1653|
|     Third| 1891|
+----------+-----+



In [15]:
# Encontrar al pokemon con ma HP

from pyspark.sql.functions import desc

max_hp_pokemon = pokemon_df.select("name", "hp").orderBy(desc("hp")).limit(1)
max_hp_pokemon.show()


+--------------+---+
|          name| hp|
+--------------+---+
|Eternatus VMAX|340|
+--------------+---+



In [19]:
# Mostrar los Pokémon legendarios:

pokemon_df.select("rarity").distinct().show()




+--------------------+
|              rarity|
+--------------------+
| put Deoxys on to...|
|[{'name': 'Upward...|
| {'name': 'Thunde...|
| 'name': 'Super P...|
| 'type': 'Pokémon...|
|          Rare Prime|
| {'name': 'Electr...|
| {'name': 'Combus...|
|[{'name': 'Chomp'...|
|"[{'name': 'Venge...|
| put Castform on ...|
| 'convertedEnergy...|
|  'cost': ['Psychic'|
|   'cost': ['Grass']|
| {'name': 'Brutal...|
|"[{'name': 'Hydro...|
| done to Seaking....|
|[{'name': 'Poison...|
| or anything else...|
|['Colorless', 'Co...|
+--------------------+
only showing top 20 rows



In [20]:
from pyspark.sql.functions import col

# Explorar la columna "rarity"
pokemon_df.select("rarity").distinct().show(truncate=False)

# Filtrar Pokémon legendarios
legendary_pokemon = pokemon_df.filter(
    (col("rarity").like("%Legend%")) |
    (col("rarity").like("%Rare Prime%")) |
    (col("rarity").like("%EX%"))  # Puedes agregar más condiciones según las rarezas de tus Pokémon legendarios
)

legendary_pokemon.select("id", "name", "rarity").show(truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|rarity                                                                                                                                                                                                                                                                                                                                                                                |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [29]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

# Función para extraer nombres de ataques y habilidades
def extract_names(rarity):
    # Utilizar expresiones regulares para extraer nombres de ataques
    attack_names = re.findall(r"'name':\s*'([^']+)'|"r'"name":\s*"([^"]+)"', str(rarity))

    # Utilizar expresiones regulares para extraer nombres de habilidades
    ability_names = re.findall(r"'abilities':\s*\[.*?{'name':\s*'([^']+)'|"r"'abilities':\s*\[.*?\"name\":\s*\"([^']+)'", str(rarity))

    # Filtrar elementos no vacíos y combinar las listas
    attack_names = [name for name in sum(attack_names, ()) if name]
    ability_names = [name for name in sum(ability_names, ()) if name]

    return attack_names, ability_names

# Registro de la función UDF
extract_names_udf = udf(extract_names, ArrayType(StringType()))

# Convertir la columna "rarity" a tipo de datos String
pokemon_df = pokemon_df.withColumn("rarity", pokemon_df["rarity"].cast(StringType()))

# Aplicar la función UDF a la columna "rarity" para extraer nombres de ataques y habilidades
pokemon_df_with_names = pokemon_df.withColumn("names", extract_names_udf("rarity"))

# Dividir el resultado en columnas separadas para ataques y habilidades
pokemon_df_with_attacks = pokemon_df_with_names.withColumn("attack_names", pokemon_df_with_names["names"].getItem(0))
pokemon_df_with_abilities = pokemon_df_with_attacks.withColumn("ability_names", pokemon_df_with_names["names"].getItem(1))

# Mostrar el resultado
pokemon_df_with_abilities.select("id", "name", "attack_names", "ability_names").show(truncate=False)


+--------+----------+--------------+-------------+
|id      |name      |attack_names  |ability_names|
+--------+----------+--------------+-------------+
|base1-1 |Alakazam  |[Confuse Ray] |[]           |
|base1-2 |Blastoise |[Hydro Pump]  |[]           |
|base1-3 |Chansey   |[]            |[]           |
|base1-4 |Charizard |[Fire Spin]   |[]           |
|base1-5 |Clefairy  |[]            |[]           |
|base1-6 |Gyarados  |[]            |[]           |
|base1-7 |Hitmonchan|[]            |[]           |
|base1-8 |Machamp   |[Seismic Toss]|[]           |
|base1-9 |Magneton  |[]            |[]           |
|base1-10|Mewtwo    |[]            |[]           |
|base1-11|Nidoking  |[]            |[]           |
|base1-12|Ninetales |[]            |[]           |
|base1-13|Poliwrath |[]            |[]           |
|base1-14|Raichu    |[]            |[]           |
|base1-15|Venusaur  |[Solarbeam]   |[]           |
|base1-16|Zapdos    |[]            |[]           |
|base1-17|Beedrill  |[]        

In [30]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

# Función para extraer nombres de ataques y habilidades
def extract_names(rarity):
    # Utilizar expresiones regulares para extraer nombres de ataques
    attack_names = re.findall(r"'name':\s*'([^']+)'|"r'"name":\s*"([^"]+)"', str(rarity))

    # Utilizar expresiones regulares para extraer nombres de habilidades
    ability_names = re.findall(r"'abilities':\s*\[.*?{'name':\s*'([^']+)'|"r"'abilities':\s*\[.*?\"name\":\s*\"([^']+)'", str(rarity))

    # Filtrar elementos no vacíos y combinar las listas
    attack_names = [name for name in sum(attack_names, ()) if name]
    ability_names = [name for name in sum(ability_names, ()) if name]

    return attack_names, ability_names

# Registro de la función UDF
extract_names_udf = udf(extract_names, ArrayType(StringType()))

# Aplicar la función UDF a la columna "rarity" para extraer nombres de ataques y habilidades
pokemon_df_with_names = pokemon_df.withColumn("names", extract_names_udf("rarity"))

# Dividir el resultado en columnas separadas para ataques y habilidades
pokemon_df_with_attacks = pokemon_df_with_names.withColumn("attack_names", pokemon_df_with_names["names"].getItem(0))
pokemon_df_with_abilities = pokemon_df_with_attacks.withColumn("ability_names", pokemon_df_with_names["names"].getItem(1))

# Mostrar el resultado
pokemon_df_with_abilities.select("id", "name", "attack_names", "ability_names").show(truncate=False)


+--------+----------+--------------+-------------+
|id      |name      |attack_names  |ability_names|
+--------+----------+--------------+-------------+
|base1-1 |Alakazam  |[Confuse Ray] |[]           |
|base1-2 |Blastoise |[Hydro Pump]  |[]           |
|base1-3 |Chansey   |[]            |[]           |
|base1-4 |Charizard |[Fire Spin]   |[]           |
|base1-5 |Clefairy  |[]            |[]           |
|base1-6 |Gyarados  |[]            |[]           |
|base1-7 |Hitmonchan|[]            |[]           |
|base1-8 |Machamp   |[Seismic Toss]|[]           |
|base1-9 |Magneton  |[]            |[]           |
|base1-10|Mewtwo    |[]            |[]           |
|base1-11|Nidoking  |[]            |[]           |
|base1-12|Ninetales |[]            |[]           |
|base1-13|Poliwrath |[]            |[]           |
|base1-14|Raichu    |[]            |[]           |
|base1-15|Venusaur  |[Solarbeam]   |[]           |
|base1-16|Zapdos    |[]            |[]           |
|base1-17|Beedrill  |[]        

In [38]:
# Extracción de información descriptiva
descriptive_info = pokemon_df.filter("rarity NOT LIKE '%name%'")  # Filtrar las filas que no contienen "name"
descriptive_info.select("id", "name", "rarity").show(truncate=False)


+--------+----------+---------------------------------------+
|id      |name      |rarity                                 |
+--------+----------+---------------------------------------+
|base1-3 |Chansey   | 'Colorless'                           |
|base1-5 |Clefairy  |['Colorless']                          |
|base1-6 |Gyarados  |Rare Holo                              |
|base1-7 |Hitmonchan|Rare Holo                              |
|base1-9 |Magneton  |Rare Holo                              |
|base1-10|Mewtwo    |['Colorless', 'Colorless', 'Colorless']|
|base1-11|Nidoking  |Rare Holo                              |
|base1-12|Ninetales |Rare Holo                              |
|base1-13|Poliwrath | 'Colorless'                           |
|base1-14|Raichu    | done to Raichu.""}                    |
|base1-16|Zapdos    |Rare Holo                              |
|base1-17|Beedrill  |Rare                                   |
|base1-18|Dragonair |Rare                                   |
|base1-1

In [40]:
# Pokémon más común en la base de datos:
common_pokemon = pokemon_df.groupBy("name").count().orderBy("count", ascending=False)
common_pokemon.show()


+----------+-----+
|      name|count|
+----------+-----+
|   Pikachu|   98|
|     Eevee|   58|
| Magnemite|   41|
|    Raichu|   41|
|Charmander|   38|
|     Unown|   36|
|   Snorlax|   35|
|  Magneton|   35|
|  Magikarp|   34|
|   Voltorb|   32|
| Electrode|   31|
| Growlithe|   31|
|    Meowth|   31|
|    Lapras|   30|
|    Vulpix|   30|
| Ninetales|   29|
|   Torchic|   28|
|  Gyarados|   28|
|  Arcanine|   28|
|   Lucario|   27|
+----------+-----+
only showing top 20 rows



In [41]:
# Número total de Pokémon por generación: CARTAS DE POKEMON
pokemon_per_generation = pokemon_df.groupBy("generation").count().orderBy("generation")
pokemon_per_generation.show()


+----------+-----+
|generation|count|
+----------+-----+
|    Eighth| 3665|
|     Fifth| 1653|
|     First|  758|
|    Fourth| 1438|
|   Fourth |  592|
|     Ninth| 1304|
|     Other|  157|
|    Second| 1031|
|   Seventh| 2973|
|     Sixth| 1710|
|     Third| 1891|
+----------+-----+



In [44]:
# Promedio de puntos de salud (HP) por generación:CARTAS
average_hp_per_generation = pokemon_df.groupBy("generation").agg({"hp": "avg"}).orderBy("generation")
average_hp_per_generation.show()


+----------+------------------+
|generation|           avg(hp)|
+----------+------------------+
|    Eighth| 142.0730503455084|
|     Fifth| 98.01078894133514|
|     First| 60.53872053872054|
|    Fourth| 81.34270101483216|
|   Fourth | 75.82857142857142|
|     Ninth|124.28317008014247|
|     Other| 68.40764331210191|
|    Second| 65.22928490351873|
|   Seventh|122.35749185667753|
|     Sixth|104.58333333333333|
|     Third| 70.38006230529595|
+----------+------------------+

